In [1]:
import json
import os
import pandas as pd
import requests
from datetime import datetime
import folium
from folium.plugins import MarkerCluster
import ast

In [2]:
def get_plznito_current_data():
    # http://plzni.to/api/1.0/tickets/list?categoryId=0&statusId=0&arch=0&term=&own=0&term=
    with requests.get('http://plzni.to/api/1.0/tickets/list?categoryId=0&statusId=0&arch=0&term=&own=0&term=') as response:
        data = response.json()
        return data
    return {}

def filter_data(data):
    data_cyklo = [x for x in data["items"] if "cykl" in x["report"].lower() or "kolob" in x["report"].lower() or "cikli" in x["report"].lower()]
    return data_cyklo

def merge_data(data_old, data_new):
    ids_new = [x["id"] for x in data_new]
    data_old = [x for x in data_old if x["id"] not in ids_new]
    data_out = data_old + data_new
    return data_out

In [4]:
# load new data
data_current = get_plznito_current_data()

# save all for later
json.dump(data_current, open(datetime.today().strftime('%Y-%m-%d-%H:%M:%S')+".json", "w"), indent=4)

In [8]:
# how to process all data
# aka DB restore
data = json.load(open("../plznito_cyklo.json"))
for i in sorted(os.listdir(".")):
    if i.endswith(".json"):
        data_new = json.load(open(i))
        data_cyklo = filter_data(data_new)
        data = merge_data(data, data_cyklo)

json.dump(data, open("../plznito_cyklo.json", "w"), indent=4)

df_new = pd.DataFrame(data)
df_new = df_new.set_index("id")
df_new


,name,investigator_id,status_id,category_id,description,solution,longitude,latitude,passed_on,passed_on_date,...,archived,zdroj,ip,address,notifi,duration,report,formatted,documents,photos
id,,,,,,,,,,,,,,,,,,,,,
17492,Větve v průjezdním a průchozím profilu,9,2,3,Do profilu smíšené stezky pro chodce a cyklist...,Kontaktní centrum - 15.6.2021\n- předáno k řeš...,13.391298986971378,49.7252900208888,7.0,"{'date': '2021-06-15 13:33:36.000000', 'timezo...",...,None,0,192.168.90.83,Plzeň 2-Slovany,,0,Do profilu smíšené stezky pro chodce a cyklist...,"{'name': 'Větve v průjezdním…', 'id': 17492, '...",[],"[{'id': 19930, 'name': 'empty', 'url': 'http:/..."
17405,Prosím o údržbu cesty parkem,9,6,10,Cesta parkem od zázemí softballového hřiště sm...,Kontaktní centrum - 11.6.2021\n- vyjádření SVS...,13.364914456283556,49.718407418196016,5.0,"{'date': '2021-06-10 13:29:34.000000', 'timezo...",...,None,1,192.168.90.83,ceske udoli 1191 301 00 plzen 3-jizni predmest...,,1,Cesta parkem od zázemí softballového hřiště sm...,"{'name': 'Prosím o údržbu…', 'id': 17405, 'des...",[],"[{'id': 19846, 'name': 'ceske.jpg', 'url': 'ht..."
17372,OZNAČENÍ,313,2,3,NĚJAK SE ZAPOMĚLO OZNAČIT ŽE K UNIVERZITĚ NA K...,Kontaktní centrum - 9.6.2021\nProsíme zadavate...,13.360183039581285,49.72680160481225,NaN,None,...,None,1,192.168.90.48,bory 301 00 plzen-plzen 3 cesko,,7,NĚJAK SE ZAPOMĚLO OZNAČIT ŽE K UNIVERZITĚ NA K...,"{'name': 'OZNAČENÍ', 'id': 17372, 'description...",[],[]
17368,TRAGICKEJ POVRCH,313,6,3,TADY FAKT NEJDE JET PO TĚCH KOSTKÁCH UŽ ANI AU...,Kontaktní centrum - 8.6.2021\nTéma řešeno napo...,13.381721177970872,49.746470088443964,NaN,None,...,None,1,192.168.90.48,prazska 31 301 00 plzen 3 cesko,,0,TADY FAKT NEJDE JET PO TĚCH KOSTKÁCH UŽ ANI AU...,"{'name': 'TRAGICKEJ POVRCH', 'id': 17368, 'des...",[],[]
17325,Nános písku a štěrku,10,3,5,V místě pro přecházení a přejezdu pro cyklisty...,Kontaktní centrum - 8.6.2021\n- vyjádření ÚMO ...,13.402451612055302,49.73230127947788,7.0,"{'date': '2021-06-07 12:16:05.000000', 'timezo...",...,None,0,192.168.90.59,Plzeň 2-Slovany,,3,V místě pro přecházení a přejezdu pro cyklisty...,"{'name': 'Nános písku a štěrku', 'id': 17325, ...",[],"[{'id': 19748, 'name': 'empty', 'url': 'http:/..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18272,Odpadky,10,6,4,odpadky u cyklostezky,Kontaktní centrum - 28.7.2021\n- předáno k řeš...,13.35727738,49.74716827,22.0,"{'date': '2021-07-28 11:13:48.000000', 'timezo...",...,None,0,192.168.90.34,Plzeň 3,,35,odpadky u cyklostezky,"{'name': 'Odpadky', 'id': 18272, 'description'...",[],"[{'id': 20841, 'name': 'empty', 'url': 'http:/..."
18252,Odstavená stříbrná Felicia,10,3,11,V prostoru křižovatky a proti výjezdu z cyklo ...,Kontaktní centrum - 8.9.2021\n- vyjádření SVSM...,13.366576084768282,49.73910877534335,5.0,"{'date': '2021-07-26 11:04:50.000000', 'timezo...",...,None,1,192.168.90.34,nam ceskych bratri 1854 301 00 plzen 3-jizni p...,,44,V prostoru křižovatky a proti výjezdu z cyklo ...,"{'name': 'Odstavená stříbrná…', 'id': 18252, '...",[],[]
18156,Díra v křižovatce,9,6,3,Křižovatka Doudlevecká x U trati. podélná díra...,Kontaktní centrum - 16.9.2021\n- vyjádření SVS...,13.379730978881822,49.74053883292208,5.0,"{'date': '2021-07-19 11:42:09.000000', 'timezo...",...,None,1,192.168.90.68,doudlevecka 8 301 00 plzen 3 cesko,,59,Křižovatka Doudlevecká x U trati. podélná díra...,"{'name': 'Díra v křižovatce', 'id': 18156, 'de...",[],"[{'id': 20694, 'name': 'dou.jpg', 'url': 'http..."


In [ ]:
# update with daily data

# filter only cyklo and convert to csv
data_cyklo_current = filter_data(data_current)

# add data to our db
data_db = json.load(open("../plznito_cyklo.json"))
data_cyklo_updated = merge_data(data_db, data_cyklo_current)
json.dump(data_cyklo_updated, open("../plznito_cyklo.json", "w"), indent=4)

In [8]:
df_new = pd.DataFrame(data_current)
df_new = df_new.set_index("id")
df_new

,name,investigator_id,status_id,category_id,description,solution,longitude,latitude,passed_on,passed_on_date,...,archived,zdroj,ip,address,notifi,duration,report,formatted,documents,photos
id,,,,,,,,,,,,,,,,,,,,,
17492,Větve v průjezdním a průchozím profilu,9,2,3,Do profilu smíšené stezky pro chodce a cyklist...,Kontaktní centrum - 15.6.2021\n- předáno k řeš...,13.391298986971378,49.7252900208888,7.0,"{'date': '2021-06-15 13:33:36.000000', 'timezo...",...,None,0,192.168.90.83,Plzeň 2-Slovany,,0,Do profilu smíšené stezky pro chodce a cyklist...,"{'name': 'Větve v průjezdním…', 'id': 17492, '...",[],"[{'id': 19930, 'name': 'empty', 'url': 'http:/..."
17405,Prosím o údržbu cesty parkem,9,6,10,Cesta parkem od zázemí softballového hřiště sm...,Kontaktní centrum - 11.6.2021\n- vyjádření SVS...,13.364914456283556,49.718407418196016,5.0,"{'date': '2021-06-10 13:29:34.000000', 'timezo...",...,None,1,192.168.90.83,ceske udoli 1191 301 00 plzen 3-jizni predmest...,,1,Cesta parkem od zázemí softballového hřiště sm...,"{'name': 'Prosím o údržbu…', 'id': 17405, 'des...",[],"[{'id': 19846, 'name': 'ceske.jpg', 'url': 'ht..."
17372,OZNAČENÍ,313,2,3,NĚJAK SE ZAPOMĚLO OZNAČIT ŽE K UNIVERZITĚ NA K...,Kontaktní centrum - 9.6.2021\nProsíme zadavate...,13.360183039581285,49.72680160481225,NaN,None,...,None,1,192.168.90.48,bory 301 00 plzen-plzen 3 cesko,,7,NĚJAK SE ZAPOMĚLO OZNAČIT ŽE K UNIVERZITĚ NA K...,"{'name': 'OZNAČENÍ', 'id': 17372, 'description...",[],[]
17368,TRAGICKEJ POVRCH,313,6,3,TADY FAKT NEJDE JET PO TĚCH KOSTKÁCH UŽ ANI AU...,Kontaktní centrum - 8.6.2021\nTéma řešeno napo...,13.381721177970872,49.746470088443964,NaN,None,...,None,1,192.168.90.48,prazska 31 301 00 plzen 3 cesko,,0,TADY FAKT NEJDE JET PO TĚCH KOSTKÁCH UŽ ANI AU...,"{'name': 'TRAGICKEJ POVRCH', 'id': 17368, 'des...",[],[]
17325,Nános písku a štěrku,10,3,5,V místě pro přecházení a přejezdu pro cyklisty...,Kontaktní centrum - 8.6.2021\n- vyjádření ÚMO ...,13.402451612055302,49.73230127947788,7.0,"{'date': '2021-06-07 12:16:05.000000', 'timezo...",...,None,0,192.168.90.59,Plzeň 2-Slovany,,3,V místě pro přecházení a přejezdu pro cyklisty...,"{'name': 'Nános písku a štěrku', 'id': 17325, ...",[],"[{'id': 19748, 'name': 'empty', 'url': 'http:/..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18599,Cyklisté v jednosměrné ulice,9,6,3,V úseku jednosměrné ulice od domu Palackého ná...,Kontaktní centrum - 23.8.2021\n- vyjádření MP:...,13.372526876628399,49.747359783531294,17.0,"{'date': '2021-08-18 10:44:06.000000', 'timezo...",...,None,0,192.168.90.68,"Palackého 38, Plzeň 3",,5,V úseku jednosměrné ulice od domu Palackého ná...,"{'name': 'Cyklisté v…', 'id': 18599, 'descript...",[],[]
18532,Díry u čertovy díry,313,2,3,"Spousta menších, avšak hlubokých děr, ve výcho...",Kontaktní centrum - 12.8.2021\n- předáno na SVSMP,13.334798613464342,49.70447118287168,5.0,"{'date': '2021-08-12 15:13:26.000000', 'timezo...",...,None,1,192.168.90.48,u certovy diry 1393 321 00 plzen 6-litice cesko,,14,"Spousta menších, avšak hlubokých děr, ve výcho...","{'name': 'Díry u čertovy díry', 'id': 18532, '...",[],"[{'id': 21139, 'name': 'certova1.jpg', 'url': ..."
18531,Neviditelná cedule se zákazy,313,2,7,"Dobrý den, po postavení kontejnerového ""městeč...",Kontaktní centrum - 12.8.2021\n- v řešení,13.387321630393968,49.7441475908369,16.0,"{'date': '2021-08-12 11:58:34.000000', 'timezo...",...,None,1,192.168.90.48,sumavska 301 00 301 00 plzen 3 cesko,,14,"Dobrý den, po postavení kontejnerového ""městeč...","{'name': 'Neviditelná cedule…', 'id': 18531, '...",[],"[{'id': 21138, 'name': '6as8c1yirq4eot0ykg31.j..."


In [9]:
def get_map(data):

    m = folium.Map(location=[49.7, 13.4], zoom_start=10)
    marker_cluster = MarkerCluster().add_to(m)

    for item in data_current:
        if item['status_id'] == 2:
            color = "orange"
        elif item['status_id'] == 3:
            color = "green"
        elif item['status_id'] == 6:
            color = "lightgreen"
        else:
            color = "red"
        date = item['created']['date']

        text = f"<b>{item['name']}</b><br>{date}<br>{item['description']}<br><br>{item['solution']}"

        try:
            photos = ast.literal_eval(item['photos'])
            if len(photos) > 0:
                text += f"<img src='{photos[0]['thumb']}'>"
        except ValueError:
            photos = []


        popup = folium.Popup(text, max_width=300,min_width=300)
        folium.Marker(
            location=[item["latitude"], item["longitude"]],
            popup=popup,
            icon=folium.Icon(color=color, icon="ok-sign"),
        ).add_to(marker_cluster)
    return m

In [6]:
m = get_map(data_current)
m